-------------------------------------------------------------------------------

CHIBO 2024

An attempt to static calculate the forces in (x,y,z) at all the suspension pickup points for various roll/jounce effects

18.12.2024: Created

03.01.2025: Outboard points calc working, moved functions to susprog.py and added loop 

05.01.2025: Added jounce updater for the inner points and included in susprog.py

08.01.2025: Added trail & scrub radius calc in loop

-------------------------------------------------------------------------------

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

import plotly.graph_objects as go
import matplotlib.pyplot as plt

import susprog                      # Python file containing the equations for the loop 

In [11]:
# Declare motions for evaluation

# Define maximum roll and jounce inputs
body_roll = -0  # Maximum roll angle in degrees
body_jounce = 0.05  # Maximum jounce in meters (5 cm = 0.05)

# Generate roll and jounce motion inputs
roll_angles = np.linspace(0, np.radians(body_roll), 2)  # 0 to max roll angle in 100 steps
jounce_displacements = np.linspace(0, body_jounce, 2)  # 0 to max jounce in 100 steps

In [12]:
# Suspension pickup points
#        x      y    z
upper_a_arm = np.array([
    [0.0015, 0.5065, 0.315],  # Outer point (converted from mm to m)
    [-0.135, 0.240, 0.220],   # Inner leading point
    [0.180, 0.255, 0.255]     # Inner trailing point
])

lower_a_arm = np.array([
    [-0.0175, 0.519, 0.137],  # Outer point
    [-0.180, 0.165, 0.087],   # Inner leading point
    [0.180, 0.170, 0.087]     # Inner trailing point
])

tie_rod = np.array([
    [0.060, 0.515, 0.160],  # Outer point
    [0.070, 0.195, 0.100]   # Inner point
])

tire_cop = np.array([-0.005, 0.525, 0])  # Converted from mm to m
force_vector = np.array([50, 500, -2000.0])  # Force at the CoP (N)


In [13]:
# Calculate lengths for upper and lower A-arms and tie rod
upper_lengths = [
    np.linalg.norm(upper_a_arm[1] - upper_a_arm[0]),  # Upper outer to inner leading
    np.linalg.norm(upper_a_arm[2] - upper_a_arm[0]),  # Upper outer to inner trailing
    np.linalg.norm(upper_a_arm[0] - lower_a_arm[0])   # Upper outer to lower outer
]

lower_lengths = [
    np.linalg.norm(lower_a_arm[1] - lower_a_arm[0]),  # Lower outer to inner leading
    np.linalg.norm(lower_a_arm[2] - lower_a_arm[0]),  # Lower outer to inner trailing
    np.linalg.norm(lower_a_arm[0] - tire_cop)         # Lower outer to tire center of pressure
]

tie_length = [
    np.linalg.norm(tie_rod[1] - tie_rod[0]),          # Tie rod outer to inner leading
    np.linalg.norm(upper_a_arm[0] - tie_rod[0]),      # Tie rod outer to upper outer
    np.linalg.norm(lower_a_arm[0] - tie_rod[0])       # Tie rod outer to lower outer
]

In [5]:
def calculate_roll_centre(upper, lower, cop):
    """
    Calculates the roll centre based on 3D geometry:
    1. Project a-arms to YZ plane (X=0) 
    2. Find equation of upper & lower lines
    3. Find SV IC
    4. Equation of line from tire_cop==>SV IC
    5. Find z when y=0
    6. Return rc_z
    """
    
    # Find equation of line in form y = m*x + c
    def line_equation(p1, p2):
        """Return slope and intercept of line through two points."""
        m = (p2[1] - p1[1]) / (p2[0] - p1[0])       # Find slope of line
        c = p1[1] - m * p1[0]                       # Find intercept of line
        return m, c
    
    # Step 1: Project arms onto front view (XZ plane)
    upper_outer_2d = upper[0, [1,2]]  # [y, z]
    upper_inner_2d = upper[1, [1,2]]  # Using front inner point
    lower_outer_2d = lower[0, [1,2]]
    lower_inner_2d = lower[1, [1,2]]
    tire_cop_2d = cop[[1, 2]]
    
    # Step 2: Find equation of line in XY plane
    m_upper, c_upper = line_equation(upper_outer_2d, upper_inner_2d)
    m_lower, c_lower = line_equation(lower_outer_2d, lower_inner_2d)
    
    # Step 3: Find SV IC
    ic_y = (c_lower - c_upper) / (m_upper - m_lower)
    ic_z = m_upper * ic_y + c_upper
    SV_IC = np.array([ic_y, ic_z])
    
    # Step 4: Find line of action of force
    m_load, rc_z = line_equation(tire_cop_2d, SV_IC)

    return rc_z

rc_z = calculate_roll_centre(upper_a_arm, lower_a_arm, tire_cop)


In [14]:
# Run simulation

# Columns: [roll_angle (rad), jounce (m), suspension_positions (1x9), rc_x, rc_y, rc_z]
results = np.zeros((len(roll_angles), 27))  # 100 rows, 36 columns
#rc, geo_roll_centre = susprog.calculate_roll_center(upper_a_arm, lower_a_arm, tire_cop)
rc_data = susprog.calculate_rc_simple(upper_a_arm, lower_a_arm, tire_cop)
rc = rc_data[0:3]

# Set rotation point to origin
# rc = [0,0,0]

# Roll axis (example: x-axis)
roll_axis = np.array([1, 0, 0])

# Initialize rotated arms as a copy of the original
upper_modified = upper_a_arm.copy()
lower_modified = lower_a_arm.copy()
tierod_modified = tie_rod.copy()

# Loop through each roll and jounce input
for i, (roll_angle, jounce_disp) in enumerate(zip(roll_angles, jounce_displacements)):
    # Step 1: Rotate and jounce the inner points of the arms and tie rod
    upper_modified = susprog.rotate_points(upper_a_arm, rc, roll_axis, roll_angle, jounce_disp)
    #upper_modified = susprog.jounce_innerpoints(upper_a_arm, upper_modified, jounce_disp)

    lower_modified = susprog.rotate_points(lower_a_arm, rc, roll_axis, roll_angle, jounce_disp)
    #lower_modified = susprog.jounce_innerpoints(lower_a_arm, lower_modified, jounce_disp)
    
    tierod_modified = susprog.rotate_points(tie_rod, rc, roll_axis, roll_angle, jounce_disp)
    #tierod_modified = susprog.jounce_innerpoints(tie_rod, tierod_modified, jounce_disp)

    # Step 3: Update outer points of lower, upper, and tie rod
    # Lower A-arm
    points_temp = np.vstack((lower_modified, tire_cop))
    lower_modified[0:1] = susprog.solve_outer_position(points_temp, lower_lengths)
    del points_temp

    # Upper A-arm
    points_temp = np.vstack((upper_modified, lower_modified[0:]))
    upper_modified[0:1] = susprog.solve_outer_position(points_temp, upper_lengths)
    del points_temp

    # Tie rod
    points_temp = np.vstack((tierod_modified, upper_modified[0:1], lower_modified[0:1]))
    tierod_modified[0:1] = susprog.solve_outer_position(points_temp, tie_length)
    del points_temp


    # Step 4: Calculate the new roll center
    rc_data = susprog.calculate_rc_simple(upper_modified, lower_modified, tire_cop)
    rc = rc_data[0:3]
    # Flatten geometry data dictionary into a list
    """geometry_values = [
        geo_roll_centre['instant_center_2d'][0],  # Instant center x
        geo_roll_centre['instant_center_2d'][1],  # Instant center z
        geo_roll_centre['swing_arm_length'],      # Swing arm length
        geo_roll_centre['upper_arm_angle'],       # Upper arm angle
        geo_roll_centre['lower_arm_angle'],       # Lower arm angle
        *geo_roll_centre['upper_normal'],         # Upper plane normal (3 components)
        *geo_roll_centre['lower_normal']          # Lower plane normal (3 components)
    ]"""
    
    trails = susprog.trail_calc(upper_modified, lower_modified, tire_cop)

    # Step 5: Store the results
    results[i, 0] = roll_angle  # Roll angle (in radians)
    results[i, 1] = jounce_disp  # Jounce displacement
    results[i, 2:11] = lower_modified.flatten()  # Lower A-arm positions (3x3 -> 1x9)
    results[i, 11:20] = upper_modified.flatten()  # Upper A-arm positions (3x3 -> 1x9)
    results[i, 20:25] = rc_data  # Roll center (1x8)
    #results[i, 23] = geo_roll_centre['instant_center_2d'][0]  # IC x
    #results[i, 24] = geo_roll_centre['instant_center_2d'][1]  # IC z
    #results[i, 25] = geo_roll_centre['swing_arm_length']      # Swing arm length
    #results[i, 26] = geo_roll_centre['upper_arm_angle']       # Upper arm angle
    #results[i, 27] = geo_roll_centre['lower_arm_angle']       # Lower arm angle
    #results[i, 28:31] = geo_roll_centre['upper_normal']       # Upper plane normal (x, y, z)
    #results[i, 31:34] = geo_roll_centre['lower_normal']       # Lower plane normal (x, y, z)
    results[i, 25:28] = trails # Steering axis==> tire cop (1x3)

del i

# Convert results to a DataFrame for easier visualization and saving
column_names = [
    "roll_angle", "jounce", 
    "lower_outer_x", "lower_outer_y", "lower_outer_z", 
    "lower_inner1_x", "lower_inner1_y", "lower_inner1_z", 
    "lower_inner2_x", "lower_inner2_y", "lower_inner2_z",
    "upper_outer_x", "upper_outer_y", "upper_outer_z", 
    "upper_inner1_x", "upper_inner1_y", "upper_inner1_z", 
    "upper_inner2_x", "upper_inner2_y", "upper_inner2_z",
    "rc_x", "rc_y", "rc_z",
    "ic_y", "ic_z", # "sal", "upper_angle", "lower_angle",  # Geometry data
    # "upper_normal_x", "upper_normal_y", "upper_normal_z",  # Upper plane normal
    # "lower_normal_x", "lower_normal_y", "lower_normal_z",   # Lower plane normal
    "Steering_trail_x", "Scrub_radius_y"
]
results_df = pd.DataFrame(results, columns=column_names)

# Save the results as a CSV file (optional)
results_df.to_csv("suspension_simulation_results.csv", index=False)

# Display the first few rows
#print(results_df.head())


In [15]:
# Extract the initial and final suspension points from the results
lower_initial = results[0, 2:11].reshape(3, 3)  # First row, lower A-arm (3x3)
upper_initial = results[0, 11:20].reshape(3, 3)  # First row, upper A-arm (3x3)

lower_final = results[-1, 2:11].reshape(3, 3)  # Last row, lower A-arm (3x3)
upper_final = results[-1, 11:20].reshape(3, 3)  # Last row, upper A-arm (3x3)

# Extract roll centers
rc_initial = results[0, 20:23]  # Initial roll center
rc_final = results[-1, 20:23]  # Final roll center

# Create a 3D scatter plot
fig = go.Figure()

# Add initial lower A-arm points
fig.add_trace(go.Scatter3d(
    x=lower_initial[:, 0], y=lower_initial[:, 1], z=lower_initial[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='blue'),
    name='Lower A-arm (Initial)'
))

# Add final lower A-arm points
fig.add_trace(go.Scatter3d(
    x=lower_final[:, 0], y=lower_final[:, 1], z=lower_final[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='red'),
    name='Lower A-arm (Final)'
))

# Add initial upper A-arm points
fig.add_trace(go.Scatter3d(
    x=upper_initial[:, 0], y=upper_initial[:, 1], z=upper_initial[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='green'),
    name='Upper A-arm (Initial)'
))

# Add final upper A-arm points
fig.add_trace(go.Scatter3d(
    x=upper_final[:, 0], y=upper_final[:, 1], z=upper_final[:, 2],
    mode='markers+lines',
    marker=dict(size=5, color='orange'),
    name='Upper A-arm (Final)'
))

# Add roll center points
fig.add_trace(go.Scatter3d(
    x=[rc_initial[0], rc_final[0]],
    y=[rc_initial[1], rc_final[1]],
    z=[rc_initial[2], rc_final[2]],
    mode='markers+lines',
    marker=dict(size=8, color='purple'),
    name='Roll Center'
))

# Configure the layout
fig.update_layout(
    scene=dict(
        xaxis_title='X-axis (m)',
        yaxis_title='Y-axis (m)',
        zaxis_title='Z-axis (m)'
    ),
    title="3D Suspension Geometry (Initial vs. Final)",
    legend=dict(x=0.1, y=0.9)
)

# Show the figure
fig.show()

# Print the initial and final points
print("Initial Lower A-arm Points:\n", lower_initial)
print("Final Lower A-arm Points:\n", lower_final)

print("Initial Upper A-arm Points:\n", upper_initial)
print("Final Upper A-arm Points:\n", upper_final)

print("Initial Roll Center:", rc_initial)
print("Final Roll Center:", rc_final)

Initial Lower A-arm Points:
 [[-0.0175  0.519   0.137 ]
 [-0.18    0.165   0.087 ]
 [ 0.18    0.17    0.087 ]]
Final Lower A-arm Points:
 [[-0.01762307  0.52252103  0.137     ]
 [-0.18        0.165       0.137     ]
 [ 0.18        0.17        0.137     ]]
Initial Upper A-arm Points:
 [[ 0.0015  0.5065  0.315 ]
 [-0.135   0.24    0.22  ]
 [ 0.18    0.255   0.255 ]]
Final Upper A-arm Points:
 [[ 0.00636311  0.5168246   0.31472097]
 [-0.135       0.24        0.27      ]
 [ 0.18        0.255       0.305     ]]
Initial Roll Center: [0.         0.         0.01061696]
Final Roll Center: [0.         0.         0.06489814]


In [8]:
# Plot RC vs roll

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["rc_y"],
        mode='lines',
        marker=dict(color='red'),
        name='Roll Centre Location, Y',
        yaxis='y'  # Explicitly associate this trace with the primary y-axis
    )
)

# Add the second trace for "rc_z" on the secondary y-axis
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["rc_z"],
        mode='lines',
        marker=dict(color='orange'),
        name='Roll Centre Location, Z',
        yaxis='y2'  # Explicitly associate this trace with the secondary y-axis
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Roll Centre Location',
    xaxis=dict(title='Roll Angle (radians)'),
    yaxis=dict(
        title='RC Location, Y (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange'),
        anchor='x',
        overlaying='y',
        side='right'  # Place the secondary y-axis on the right side
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [16]:
# Plot RC vs jounce

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=results_df["jounce"],
        y=results_df["rc_z"],
        mode='lines',
        marker=dict(color='red'),
        name='Roll Centre Location, z',
        yaxis='y'  # Explicitly associate this trace with the primary y-axis
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Roll Centre Location',
    xaxis=dict(title='Jounce (m)'),
    yaxis=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()
